[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rndsrc/2024_nobel-phys_hs/blob/main/Hopfield.ipynb)

# Exploring the 2024 Nobel Prize in Physics

## 0. Setup (5 min)

### Access the Notebook

To start working with this notebook, open it using one of the following methods:

- [**GitHub Codespaces**](https://github.com/features/codespaces): If you are using a GitHub repository, you can launch a Codespace, which provides an online VS Code environment with Jupyter support.
- [**Google Colab**](https://colab.research.google.com/): If you prefer a cloud-based Python environment, open this notebook in Google Colab.
- **Local Jupyter Lab:** If you have Jupyter installed on your machine, run the following command in your terminal: `jupyter lab`

### Quick Tour of Jupyter Notebook

Jupyter Notebook consists of **cells** that can contain either **Markdown** (formatted text) or **Python code**.

- **Markdown Cells:** Used for documentation and instructions.
  Double-click a cell to edit it, and press `Shift + Enter` to render it.
- **Code Cells:** Used to write and execute Python code.
  Click inside a cell and press `Shift + Enter` to run it.

### Example: Run Your First Python Code

Try running the following code cell to check if Python is working correctly:

In [ ]:
# Print a testing welcome message

def greet():
    return "Welcome to the Hopfield Network Notebook!"

print(greet())

### Installing Dependencies (if needed)

In a Jupyter Notebook, an `!` (exclamation mark) is used to execute shell (command-line) commands directly from a code cell.
This allows users to run commands as they would in a terminal, without leaving the notebook.
If running this notebook locally, ensure you have the required libraries installed:

In [ ]:
# "Shell out" to the operating system to install the necessary packages

! pip install numpy matplotlib

### Define helper functions

To make it more fun, we will create animation and work with images along the way.
These are plotting details that we don't need to spend time on.
Just run the following cell to make sure the `animate()` and `load()` functions are defined.

In [ ]:
# Define `animate(state, Nsub=100, N=250)` to create animation and `load(filename)` to load image files

from matplotlib import pyplot as plt
from matplotlib import image  as img
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

import numpy as np
from   numpy.random import randint

def animate(state, update, nsub=100, n=250, cmap='viridis'):
    fig, ax = plt.subplots(1, 1, figsize=(5,5))
    im = ax.imshow(state, cmap=cmap)
    def func(i):
        update(state, nsub)
        im.set_array(state)
        return [im]
    an = FuncAnimation(fig, func, frames=n, interval=40, blit=True)
    plt.close()
    return an

def load(filename):
    im = img.imread(filename)
    if im.ndim == 3:
        im = np.mean(im, axis=-1)   # Convert to grayscale if needed
    im = np.where(im < 128, -1, 1)  # Convert grayscale to binary (-1, 1)
    return im

This concludes the setup!
Now, let's dive into Hopfield networks and their connection to the Ising model.

## 1. Introduction (5 min)

### What is the 2024 Nobel Prize in Physics About?

In 2024, the **Nobel Prize in Physics** was awarded to **John J. Hopfield** and **Geoffrey E. Hinton** for their pioneering work on neural networks.
Their contributions have played a significant role in the development of modern artificial intelligence and computational models inspired by biological learning mechanisms.

- **John J. Hopfield** introduced the Hopfield network, a type of recurrent neural network that models associative memory and collective computation in neural systems.
- **Geoffrey E. Hinton** made significant advancements in deep learning, particularly in training deep neural networks through the backpropagation algorithm.

Their research has had profound implications in artificial intelligence, neuroscience, and computational physics.

### Why Do Neural Networks Matter?

Neural networks are now an essential part of many everyday technologies:

- **Facial Recognition:** Used to unlock smartphones and enhance security.
- **Chatbots:** Power customer service agents that interact with users.
- **Voice Assistants:** Siri, Alexa, and Google Assistant rely on neural networks for speech recognition.
- **Self-Driving Cars:** Autonomous vehicles process sensor data using deep learning models.

## Importance of This Discovery

Neural networks have revolutionized multiple domains, including:
- **Language Translation:** Real-time translation services are powered by deep learning models.
- **Medical Diagnosis:** AI-assisted analysis of medical images improves diagnostic accuracy.
- **Scientific Research:** AI-driven simulations enhance modeling in physics, chemistry, and biology.

## 2. The Ising Model (15 min)

### What Is the Ising Model?

To understand how neural networks operate, we first introduce the **Ising model**.

The **Ising model** is a mathematical model in statistical mechanics, introduced by Wilhelm Lenz in 1920 and solved for the one-dimensional case by his student Ernst Ising in 1925.
The model was originally used to explain ferromagnetism, where magnetic materials exhibit spontaneous magnetization due to interactions between neighboring atomic spins.

- Wilhelm Lenz conceived the model as a simplified representation of magnetic interactions in a lattice, where spins can either point "up" (+1) or "down" (-1).
- Ernst Ising solved the one-dimensional version of the model in his doctoral thesis, showing that it did not exhibit phase transitions—a result that was surprising at the time.
- Lars Onsager solved the two-dimensional version of the model in 1944, demonstrating that it undergoes a phase transition at a critical temperature, where spontaneous magnetization occurs.

Since then, the Ising model has become one of the most widely studied models in statistical physics and beyond.
Its applications extend not only to physics (such as in magnetism and lattice gases) but also to fields like biology (neural networks), computer science (optimization problems), and even sociology (modeling opinion dynamics).

The **Metropolis algorithm** (1953) was developed for Monte Carlo simulations of systems like the Ising model, enabling the study of large, complex systems by simulating their thermal fluctuations and statistical properties.
This method revolutionized computational physics and remains a powerful tool in many areas of research today.

#### Basic Idea

- Consider a **grid (lattice)** where each point represents a **spin**.
- Each spin can take one of two states: **up (+1) or down (-1)**.
- Spins interact with their neighbors, aiming to align in a way that minimizes system energy.
- The system's **energy parameter** determines the likelihood of spins flipping, making temperature a crucial factor in system behavior.

Let's start programming as we introduce the different concepts.
Here, we create a python class to keep track of the state and temperature of an Ising Model.

In [ ]:
# Subclass a numpy array to store the state of an Ising Model

class Ising(np.ndarray):

    def __new__(cls, shape=(64,64), kT=1, seed=None):
        if seed is not None:
            np.random.seed(seed)

        obj = np.random.choice([-1,1], size=shape).view(cls)
        obj.kT = kT
        return obj

    def __array_finalize__(self, obj):
        if obj is None: return
        self.kT = getattr(obj, 'kT', None)

With the class `Ising`, we can now instanize a Ising model by creating a 2D grid of spin state:
```python
ising = Ising(seed=42)
```

In [ ]:
# HANDSON: instanize a Ising model by creating a 2D grid of spin state.



We can visualize the spin state by plotting `ising` as an image:
```python
plt.imshow(ising)
```
You may try out different [colormaps](https://matplotlib.org/stable/users/explain/colors/colormaps.html) by specifying the `cmap` keyword, e.g.,
```python
plt.imshow(ising, cmap='coolwarm')
```

In [ ]:
# HANDSON: visualize the spin state by plotting the state as an image



#### Interaction and Energy Minimization

- The system's total energy is determined by how well-aligned the spins are with their neighbors.
- At **high temperatures**, spins flip frequently due to thermal fluctuations, leading to disorder.
- At **low temperatures**, spins align into ordered configurations, exhibiting phase transitions similar to those in magnetic materials.
- The **Hamiltonian** (energy function) for the Ising model is:
  $$
  H = - \sum_{i,j} J_{ij} s_i s_j
  $$
  where:
  - $s_i$ represents the spin at site $i$.
  - $J_{ij}$ is the coupling strength between neighboring spins.
- The system evolves dynamically to minimize $H$, leading to patterns of stable configurations.

#### Energy Change Due to Spin Flip

When a single spin $s_i$ flips, the change in energy is calculated by comparing the energy before and after the flip.
The energy difference $\Delta E$ due to flipping the spin at site $i$ is:
$$
\Delta E = 2 s_i \sum_{j} J_{ij} s_j,
$$
where the sum is over the nearest neighbors $j$ of site $i$.
The factor of 2 arises because flipping the spin at site $i$ changes its contribution to the energy from $-s_i s_j$ to $+s_i s_j$.

Choose $J_{ij}$ so that only neighborhood cells have interaction 1, we can implement the following function:
```python
def energy_change(ising, i, j):
    I, J = ising.shape  # Get lattice dimensions
    spin = ising[i, j]  # Current spin value
    
    # Neighboring spins (periodic boundary conditions)
    neighbors = (  ising[(i+1)%I, j] 
                 + ising[(i-1)%I, j]
                 + ising[i, (j+1)%J]
                 + ising[i, (j-1)%J])

    # Energy difference due to spin flip
    return 2 * spin * neighbors
```

In [ ]:
# HANDSON: please fill out the blanks ___ of the following function

def energy_change(ising, i, j):
    ___

We may now obtain the energy change for flipping a spin at different cells:
```python
energy_change(ising, 1, 1)
```

In [ ]:
# HANDSON: choose different i and j and compute the energy change by calling `energy_change(state, i, j)`



### Metropolis Algorithm

To simulate the evolution of the system at a given temperature, we use the Metropolis algorithm.
This algorithm probabilistically accepts or rejects a spin flip based on the energy change $\Delta E$ and the temperature $T$.
The probability of accepting a spin flip is given by the Boltzmann factor:
$$
P(\text{flip}) =
\begin{cases}
1 & \text{if } \Delta E < 0, \\
\exp\left(-\frac{\Delta E}{k T}\right) & \text{if } \Delta E \geq 0,
\end{cases}
$$
where:
- $\Delta E$ is the energy change caused by the flip.
- $k$ is the Boltzmann constant.
 $T$ is the temperature.

This allows the system to "explore" higher energy states at higher temperatures (thermal fluctuations), while favoring low-energy configurations as the system cools down.

We can implement a function `flip(ising, i, j)` to determent if we need to flip the spin `ising` at grid `i`, `j`:
```python
def flip(ising, i, j):
    dE = energy_change(ising, i, j)  # Compute energy change
    
    if dE < 0:  # Check the sign of energy change
        return True  # Always accept flip if energy decreases
    else:
        return np.random.rand() < np.exp(-dE / ising.kT)  # Accept flip with Boltzmann probability
```

In [ ]:
# HANDSON: please fill out the blanks ___ of the following function

def flip(ising, i, j):
    ___

Note the `np.random.rand()` function in `flip()`.
Choose a cell with positive energy change and run flip multiple times.
What do you see?
```python
for _ in range(100):
    print(flip(ising, 1, 1))
```

In [ ]:
# HANDSON: try repeat flip() multiple times



What happen if you change the temperature?
```python
ising1 = Ising(kT=10, seed=42)
for _ in range(100):
    print(flip(ising1, 1, 1))

ising2 = Ising(kT=0.1, seed=42)
for _ in range(100):
    print(flip(ising2, 1, 1))
```

In [ ]:
# HANDSON: try change `kT` and then repeat flip()



With the above helper functions, we are ready to implement the Ising Model using the metropolis algorithm:
```python
def run(ising, N):
    for _ in range(N):  # Repeat the simulation N times
        i = randint(0, ising.shape[0])  # Random row index
        j = randint(0, ising.shape[1])  # Random column index
        if flip(ising, i, j):  # Check if we need to flip the spin
            ising[i, j] *= -1  # Actually flip the spin
```

In [ ]:
# HANDSON: please fill out the blanks ___ of the following function

def run(ising, N):
    ___

That's it!
We can how compare the spin state before
```python
plt.imshow(ising)
```
and after the simulation:
```python
run(ising, 100_000)
plt.imshow(ising)
```

In [ ]:
# HANDSON: visualize the initial spin state



In [ ]:
# HANDSON: run the simuluation for, e.g., 100,000 steps, and then visualize the final spin state



We've created two additional states above with different temperature.
How do the spin states look like after simulations?

In [ ]:
# HANDSON: run the simuluation for spin states with different temperature; visualize the final spin state



### Animate the Ising Model

Just for fun, we can now combine the implemented Ising model with the `animate(state, update, nsub, n)` helper function that we defined at the very beginning.

The `run()` function that we implemented actually has the same call signature as the `update()` function.
Hence, we can use it to create the animation:
```python
ising = Ising()
an = animate(ising, run)
```

In [ ]:
# HANDSON: use the animate(state, update, nsub, n) to create an animation object `an`



Finally, we can show the animation as a movie:
```python
HTML(an.to_html5_video())
```

In [ ]:
# HANDSON: show the animation object as a video



### Why It Matters

#### Real-World Example: Magnets and Decision-Making
- In **magnetism**, atomic spins align to form magnetic domains, illustrating how interactions shape large-scale behavior.
- In **social dynamics**, human decision-making can resemble an Ising model, where individuals adjust their opinions based on peer influence.

#### Optimization
- Many complex optimization problems, including **image recognition and combinatorial search problems**, can be formulated using the Ising model framework.
- The process of **energy minimization** in the Ising model is analogous to **error minimization** in machine learning.
- Neural networks, particularly **Hopfield networks** that we will present later, use a similar principle to find stable memory states.

## 3. Hopfield Networks (20 min)

### Memory Machines

The **Hopfield network**, introduced by **John Hopfield in 1982**, is a recurrent neural network that stores and recalls patterns through energy minimization.

- **Pattern Storage and Recall:** The network can store multiple patterns as stable configurations.
- **Robustness to Noise:** If an input is noisy or incomplete, the network still retrieves the correct stored pattern.
- **Content-Addressable Memory:** Unlike conventional memory, which retrieves data using addresses, Hopfield networks retrieve patterns based on similarity.

### Energy Landscape

The Hopfield network shares deep similarities with the Ising model:
- **Neurons replace spins**, taking binary values $+1$ or $-1$.
- **Connections between neurons** correspond to interactions between spins.
- **The system evolves toward stable, low-energy states**, just like in the Ising model.
- **Both exhibit emergent global order** despite being governed by local interactions.

The energy function of the Hopfield network is given by:
$$
E = - \sum_{i,j} W_{ij} \sigma_i \sigma_j,
$$
where $W_{ij}$ represents synaptic weights and $\sigma_i$ is the neuron state.

When a neuron updates, its **energy change** follows:
$$
\Delta E = 2 \sigma_i \sum_j W_{ij} \sigma_j.
$$

### Why It Matters

The Hopfield network played a **foundational role in artificial intelligence** by introducing energy-based models:
- Inspired later developments such as **Boltzmann machines** and **Deep Learning architectures**.
- Showed how memory and pattern recognition could emerge from simple neuron interactions.
- Provided insights into **biological neural networks**, helping us understand how the brain processes and retrieves information.

Because of the similar between a Hopfield network and Ising model, similar, we subclass a numpy array to store the state of a Hopfield network.
Instead of keeping track of the model's temperature `kT`, we create an empty array `W` to store the synaptic weights.

In [ ]:
# Subclass a numpy array to store the state of a Hopfield network

class Hopfield(np.ndarray):

    def __new__(cls, shape=(64,64), seed=None):
        N = shape[0] * shape[1]
        if seed is not None:
            np.random.seed(seed)

        obj   = np.random.choice([-1,1], size=shape).view(cls)
        obj.W = np.zeros((N, N))
        return obj

    def __array_finalize__(self, obj):
        if obj is None: return
        self.W = getattr(obj, 'W', None)

By default, an instance of the Hopfield class is random.
Let's instanize one and visualize it:
```python
hopfield = Hopfield()
plt.imshow(hopfield, cmap='gray')
```

In [ ]:
# HANDSON: instanize a Hopfield network and visualize its initial random state



### Hebbian Learning Rule

The weights in a Hopfield network are learned using the Hebbian learning rule, which strengthens the connections between neurons that are activated together.
It is given by:
$$
W_{ij} = \frac{1}{P} \sum_p \sigma_i^{(p)} \sigma_j^{(p)}.
$$
Where:
* $N$ is the number of neurons.
* $P$ is the number of patterns.
* $\sigma_i^{(p)}$ is the state of neuron $i$ in a pattern $p$.

This makes Hopfield networks **powerful for associative memory**, allowing pattern retrieval even with noise.

Using `numpy`'s function, it is straightforward to implement Hebbian's learning rule:
```python
def learn(hopfield, patterns):
    for pattern in patterns:  # Hopfield network can learn multiple patterns
        assert pattern.shape == hopfield.shape  # Ensure pattern shape matches the state shape
        p = pattern.flatten()
        hopfield.W += np.outer(p, p)  # The Hebbian Learning Rule is mathematically an outer product
        
    np.fill_diagonal(hopfield.W, 0)  # Ensure no neuron connects to itself
    hopfield.W /= len(patterns)      # Normalize by the number of patterns
```

In [ ]:
# HANDSON: please fill out the blanks ___ of the following function

def learn(hopfield, patterns):
    ___

To test it out, let's download some sample images and read them with the `load()` function that we defined at the beginning.
Because downloading images does not affect our understanding of Hopfield network, let's just run the following code cells directly:

In [ ]:
# Download images from GitHub in case this notebook is opened in Google colab

url  = "https://raw.githubusercontent.com/rndsrc/2024_nobel-phys_hs/refs/heads/main/images/"
url1 = url + "AI.jpg"
url2 = url + "cloud.jpg"
url3 = url + "computer.jpg"

! if [ ! -d images ]; then mkdir images && cd images && wget {url1} && wget {url2} && wget {url3}; fi

In [ ]:
# Load images and display them

im1 = load("images/AI.jpg")
im2 = load("images/cloud.jpg")

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(im1, cmap='gray')
ax0.set_title('Pattern "AI"')
ax1.imshow(im2, cmap='gray')
ax1.set_title('Pattern "cloud"')
plt.show()

With these sample images, let's make our Hopfield network learn from these patterns.
Note that the state of the Hopfield network has not change.
It is still random.
```python
learn(hopfield, [im1, im2])
plt.imshow(hopfield, cmap='gray')
```

In [ ]:
# HANDSON: check what happen to the Hopfield state after learning the patterns



Once the patterns are learn, we can recall each pixel value by:
\begin{align}
  \sigma_i^{(p)} = \text{sgn}\left(\sum_j W_{ij} \sigma_j^{(p)}\right)
\end{align}
which can be implemented as:
```python
def recall(hopfield, N):
    for _ in range(N):
        i = randint(0, hopfield.shape[0])  # Random row index
        j = randint(0, hopfield.shape[1])  # Random column index
        
        # Update a single pixel in the pattern
        index = i * hopfield.shape[1] + j
        hopfield[i, j] = np.sign(np.dot(hopfield.W[index], hopfield.flatten()))
```

In [ ]:
# HANDSON: please fill out the blanks ___ of the following function

def recall(hopfield, N):
    ___

That's it!
We've implemented all the functions of the Hopfield network!
We can "recall" its memory and visulize the result by:
```python
recall(hopfield, 100_000)
plt.imshow(hopfield, cmap='gray')
```

In [ ]:
# HANDSON: recall a pattern and visualize the result



### Animate the Hopfield Network

Just for fun again, we can now combine the implemented Hopfield network with the `animate(state, update, nsub, n)` helper function that we defined at the very beginning.

The `recall()` function that we implemented actually has the same call signature as the `update()` function.
Hence, we can use it to create the animation:
```python
hopfield = Hopfield()
learn(hopfield, [im1, im2])
an = animate(hopfield, recall, cmap='gray')
HTML(an.to_html5_video())
```

In [ ]:
# HANDSON: animate memory recall of a Hopfield network



## 4. Discussion (5 min)

### Closing Questions

This notebook has demonstrated how **concepts from physics and AI intersect** through models like the Ising Model and Hopfield Networks.
Here are some questions to encourage further exploration:

- How do energy minimization principles in physics apply to AI and machine learning?
- Can you think of other real-world phenomena that could be modeled using energy-based systems?
- How might AI benefit from further insights from statistical mechanics and physics?
- What improvements could be made to the Hopfield Network to make it more efficient?

For further study, students can explore more advanced models such as Boltzmann Machines, Restricted Boltzmann Networks, and Deep Learning frameworks.

### Next Steps

- Experiment with different **patterns and noise levels** in the Hopfield Network.
- Modify the **weight update rules** to observe how the network behaves.
- Implement more **complex datasets** and evaluate performance.
- Use **Jupyter Notebooks** as a tool to explore AI and physics simulations further.

This discussion wraps up our **Exploring the 2024 Nobel Prize in Physics**.
We hope this notebook inspires curiosity and further learning!